In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [3]:
data = load_wine()
X = data.data
y = data.target
dataset_name = "Wine"

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [4]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC()
}


In [5]:
results = []

def evaluate_model(name, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    results.append({"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1 Score": f1})
    print(f"\n{name} Performance:")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1 Score: {f1:.4f}")


In [6]:
for name, model in models.items():
    evaluate_model(name, model)



Logistic Regression Performance:
Accuracy: 0.9815, Precision: 0.9778, Recall: 0.9841, F1 Score: 0.9804

Random Forest Performance:
Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000

SVM Performance:
Accuracy: 0.9815, Precision: 0.9848, Recall: 0.9762, F1 Score: 0.9799


In [7]:
logreg_params = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}
grid_logreg = GridSearchCV(LogisticRegression(max_iter=200), logreg_params, cv=5, scoring='f1_macro')
grid_logreg.fit(X_train, y_train)
print("\nBest Logistic Regression Parameters:", grid_logreg.best_params_)
evaluate_model("Tuned Logistic Regression", grid_logreg.best_estimator_)



Best Logistic Regression Parameters: {'C': 0.1, 'solver': 'lbfgs'}

Tuned Logistic Regression Performance:
Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000


In [8]:
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}
grid_rf = GridSearchCV(RandomForestClassifier(), rf_params, cv=5, scoring='f1_macro')
grid_rf.fit(X_train, y_train)
print("\nBest Random Forest Parameters:", grid_rf.best_params_)
evaluate_model("Tuned Random Forest", grid_rf.best_estimator_)



Best Random Forest Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}

Tuned Random Forest Performance:
Accuracy: 0.9630, Precision: 0.9611, Recall: 0.9683, F1 Score: 0.9633


In [9]:
svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}
rand_svm = RandomizedSearchCV(SVC(), svm_params, n_iter=5, cv=5, scoring='f1_macro', random_state=42)
rand_svm.fit(X_train, y_train)
print("\nBest SVM Parameters:", rand_svm.best_params_)
evaluate_model("Tuned SVM", rand_svm.best_estimator_)



Best SVM Parameters: {'kernel': 'linear', 'gamma': 'auto', 'C': 10}

Tuned SVM Performance:
Accuracy: 0.9815, Precision: 0.9778, Recall: 0.9841, F1 Score: 0.9804


In [10]:
print("\n📊 Final Model Performance Comparison on Wine Dataset:")
df_results = pd.DataFrame(results)
print(df_results.sort_values(by="F1 Score", ascending=False).reset_index(drop=True))



📊 Final Model Performance Comparison on Wine Dataset:
                       Model  Accuracy  Precision    Recall  F1 Score
0              Random Forest  1.000000   1.000000  1.000000  1.000000
1  Tuned Logistic Regression  1.000000   1.000000  1.000000  1.000000
2        Logistic Regression  0.981481   0.977778  0.984127  0.980376
3                  Tuned SVM  0.981481   0.977778  0.984127  0.980376
4                        SVM  0.981481   0.984848  0.976190  0.979902
5        Tuned Random Forest  0.962963   0.961111  0.968254  0.963292
